**Pertemuan 3**

In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings("ignore")

**Dataset**

- membuat data set transaksi sebagai dataframe
- cek dataset transaksi

In [3]:
df = pd.DataFrame(
    
    {
    "ID Transaksi" : pd.Series([1,2,3,4,5]),
    "Item" : pd.Series([["Susu", "Roti", "Biskuit", "Mentega"],\
                       ["Susu", "Biskuit", "Gula", "Kopi"],\
                        ["Gula", "Kopi", "Susu"], ["Kopi", "Biskuit", "Susu", "Gula"],\
                            ["Mentega", "Roti", "Kopi"]]),
    }

)

In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID Transaksi  5 non-null      int64 
 1   Item          5 non-null      object
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


In [5]:
df = df.explode('Item')
df

,ID Transaksi,Item
0,1,Susu
0,1,Roti
0,1,Biskuit
0,1,Mentega
1,2,Susu
1,2,Biskuit
1,2,Gula
1,2,Kopi
2,3,Gula
2,3,Kopi


In [11]:
basket = (df.groupby(["ID Transaksi", "Item"]) ["Item"].count()\
          .unstack().reset_index().fillna(0)\
            .set_index("ID Transaksi"))

basket.head()

Item,Biskuit,Gula,Kopi,Mentega,Roti,Susu
ID Transaksi,,,,,,
1,1.0,0.0,0.0,1.0,1.0,1.0
2,1.0,1.0,1.0,0.0,0.0,1.0
3,0.0,1.0,1.0,0.0,0.0,1.0
4,1.0,1.0,1.0,0.0,0.0,1.0
5,0.0,0.0,1.0,1.0,1.0,0.0


**Algoritma Apriori**

- Algoritma Apriori ditujukan untuk mengidentifikasi item-item yang sering muncul bersama dalam suatu transaksi atau kumpulan data

- Algoritma Apriori sangat umum digunakan dalam analisis keranjang belanja, sistem rekomendasi, dan pemahaman pada konsumen

- Eksekusi algoritma apriori menggunakan libary mlextend dengan fungsi apriori

- Langkah-langkah umum apriori : 
    1. Mengeliminasi itemset berdasarkan minimum support (threshold)

    2. Menghitung confidence sejumlah perulangan tertentu
    
    3. Menentukan cross product ata kombinasi item-item yang muncul pada suatu transaksi

In [7]:
df_encode = df.groupby("Item")["Item"].count().reset_index(name = "Count")
df_encode

,Item,Count
0,Biskuit,3
1,Gula,3
2,Kopi,4
3,Mentega,2
4,Roti,2
5,Susu,4


Apriori Tahap 1

Eliminasi itemset berdasarkan minimum support dan lakukan iterasi

Iterasi I

- Hitung support setiap item

- Asumsikan minimum support 3 atau 60% (muncul 3x dari 5 transkasi)

- Filter item yang tidak memenuhi minimum support

In [7]:
def create_support(x) :
    #jumlah transaksi per item
    N = 5 # Jumlah semua transaksi
    y = (x/N)*100 #hitung presentase
    return y

In [10]:
df_i1 = df_encode.copy()
df_i1["Support"] = df_i1["Count"].apply(lambda x : create_support(x))
df_i1

,Item,Count,Support
0,Biskuit,3,60.0
1,Gula,3,60.0
2,Kopi,4,80.0
3,Mentega,2,40.0
4,Roti,2,40.0
5,Susu,4,80.0


In [13]:
df_i1 = df_i1[df_i1["Support"] >= 60]
df_i1
    

,Item,Count,Support
0,Biskuit,3,60.0
1,Gula,3,60.0
2,Kopi,4,80.0
5,Susu,4,80.0


Iterasi II

- Kombinasi setiap item menjadi 2-Itemset, hiraukan untuk item yang sudah difilter

- lalu filter kembali untuk 2-Itemset yang tidak mememnuhi minimum support

In [8]:
filtered = ["Roti", "Mentega"]
df_filtered  = df[~df["Item"].isin(filtered)]
df_encode_2 = df_filtered.groupby("Item")["Item"].count().reset_index(name = "Count")
df_i2 = df_encode_2.copy()
df_i2["Support"] = df_i2["Count"].apply(lambda x : create_support(x))
df_i2


,Item,Count,Support
0,Biskuit,3,60.0
1,Gula,3,60.0
2,Kopi,4,80.0
3,Susu,4,80.0


In [9]:
df_i2 = df_i2[df_i2["Support"] >= 60]
df_i2

,Item,Count,Support
0,Biskuit,3,60.0
1,Gula,3,60.0
2,Kopi,4,80.0
3,Susu,4,80.0


Apriori Tahap 2

Perhitungan confidence

Kombinasi

Cross pruduct atau hasil kombinasi dari item yang sering muncul dalam satu transaksi

In [10]:
basket_2 = (df_filtered.groupby(["ID Transaksi", "Item"])["Item"].count()\
            .unstack().reset_index().fillna(0)\
                .set_index("ID Transaksi"))

frequent_itemsets_2 = apriori(basket_2, min_support=0.6, use_colnames=True)
frequent_itemsets_2

,support,itemsets
0,0.6,(Biskuit)
1,0.6,(Gula)
2,0.8,(Kopi)
3,0.8,(Susu)
4,0.6,"(Biskuit, Susu)"
5,0.6,"(Gula, Kopi)"
6,0.6,"(Gula, Susu)"
7,0.6,"(Susu, Kopi)"
8,0.6,"(Gula, Susu, Kopi)"


In [11]:
rules = association_rules(frequent_itemsets_2, metric="confidence", min_threshold=0.6)
rules[["antecedents", "consequents", "support", "confidence"]]

,antecedents,consequents,support,confidence
0,(Biskuit),(Susu),0.6,1.00
1,(Susu),(Biskuit),0.6,0.75
2,(Gula),(Kopi),0.6,1.00
3,(Kopi),(Gula),0.6,0.75
4,(Gula),(Susu),0.6,1.00
5,(Susu),(Gula),0.6,0.75
6,(Susu),(Kopi),0.6,0.75
7,(Kopi),(Susu),0.6,0.75
8,"(Gula, Susu)",(Kopi),0.6,1.00
9,"(Gula, Kopi)",(Susu),0.6,1.00
